In [1]:
!pip install nltk==3.8.1

import nltk
from nltk.tokenize import sent_tokenize

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

In [2]:
from sentence_transformers import util

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Sakalti/gpt-neox-small")
model = AutoModelForCausalLM.from_pretrained("Sakalti/gpt-neox-small")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [25]:
import json

filename = 'data.json'

with open(filename, 'r', encoding='utf-8') as f:
    dataset = json.load(f)

In [26]:
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
import numpy as np

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({"pad_token": "[PAD]"})
    model.resize_token_embeddings(len(tokenizer))

model.config.pad_token_id = tokenizer.pad_token_id

texts = [d['text'] for d in dataset]

batch_size = 128

batches = [texts[i:i+batch_size] for i in range(0, len(texts), batch_size)]

document_embeddings = []
for batch in tqdm(batches):
    batch_embeddings = embedding_model.encode(batch)
    document_embeddings.extend(batch_embeddings)

document_embeddings = np.array(document_embeddings)

  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
!pip install faiss-cpu

In [27]:
import faiss

faiss.normalize_L2(document_embeddings)

dim = document_embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(document_embeddings)

In [42]:
def retrieve_relevant_docs(query, k=5):
    query_embedding = embedding_model.encode([query])
    print('query_embedding')
    faiss.normalize_L2(query_embedding)
    print('faiss.normalize_L2')
    distances, indices = index.search(query_embedding, k)
    print('Дистанции и индексы нашел')

    relevant_docs = []
    for idx in indices[0]:
        doc = dataset[idx.item()]
        relevant_docs.append(doc)

    return relevant_docs

In [29]:
def select_best_sentences(relevant_docs, query):
    best_sentences = []
    for doc in relevant_docs:
        sentences = sent_tokenize(doc['text'])
        print('Документ разделил')
        scores = [
            util.cos_sim(embedding_model.encode(query), embedding_model.encode(sent)).item()
            for sent in sentences
        ]
        print('Счет посчитал')
        best_sentence_idx = np.argmax(scores)
        best_sentences.append(sentences[best_sentence_idx])

    return best_sentences

In [35]:
def select_best_sentences_fast(relevant_docs, query):
    best_sentences = []
    print(f'Кол-во документов для анализа: {len(relevant_docs)}')
    for doc in relevant_docs:
        sentences = sent_tokenize(doc['text'])
        print('Документ разделил')
        encoded_query = embedding_model.encode([query])[0]
        encoded_sentences = embedding_model.encode(sentences)
        print('Эмбеддинги закодировал')
        similarities = util.cos_sim(encoded_query, encoded_sentences)
        print('Похожесть посчитал')
        best_sentence_idx = np.argmax(similarities)
        print('Лучшее предложение отыскал')
        best_sentences.append(sentences[best_sentence_idx])

    return best_sentences

In [38]:
def rag_generate(query):
    relevant_docs = retrieve_relevant_docs(query)
    print('Документы подобрал')

    best_sentences = select_best_sentences_fast(relevant_docs, query)
    print('Предложения выбрал')

    context = '\n'.join(best_sentences)

    input_text = f"question: {query}\ncontext: {context}"

    inputs = tokenizer(input_text, return_tensors="pt")
    outputs = model.generate(inputs.input_ids, max_new_tokens=50)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return answer.strip()

In [43]:
query = "Чем лечить фузариоз"

final_answer = rag_generate(query)

print(final_answer)

query_embedding
faiss.normalize_L2
Дистанции и индексы нашел
Документы подобрал
Кол-во документов для анализа: 10
Документ разделил
Эмбеддинги закодирова
Похожесть посчитал
Лучшее предложение отыскал
Документ разделил
Эмбеддинги закодирова
Похожесть посчитал
Лучшее предложение отыскал
Документ разделил
Эмбеддинги закодирова
Похожесть посчитал
Лучшее предложение отыскал
Документ разделил
Эмбеддинги закодирова
Похожесть посчитал
Лучшее предложение отыскал
Документ разделил
Эмбеддинги закодирова
Похожесть посчитал
Лучшее предложение отыскал
Документ разделил
Эмбеддинги закодирова
Похожесть посчитал
Лучшее предложение отыскал
Документ разделил
Эмбеддинги закодирова
Похожесть посчитал
Лучшее предложение отыскал
Документ разделил
Эмбеддинги закодирова
Похожесть посчитал
Лучшее предложение отыскал
Документ разделил
Эмбеддинги закодирова
Похожесть посчитал
Лучшее предложение отыскал
Документ разделил


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


Эмбеддинги закодирова
Похожесть посчитал
Лучшее предложение отыскал
Предложения выбрал
question: [UNK]ем лечить фузариоз context: Лечат растения на ранних стадиях болезни фунгицидами. Прекратите подкармливать алоказию в фазе покоя. Нанесите его из пульверизатора на листья, а затем аккуратно протрите листья до блеска. [UNK]е легко получить и также легко с пользой применять. Легко размножается черенками. [UNK]е легко получить и также легко с пользой применять. [UNK]е легко получить и также легко с пользой применять. [UNK]е легко получить и также легко с пользой применять. [UNK]е легко получить и также легко с пользой применять. [UNK]е легко получить и также легко с пользой применять. [UNK] [UNK]([UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UNK]、[UN